# TERA - Aula 27
## Clustering

Objetivos gerais de algoritmos de clustering:
- Análise exploratória dos dados
- Encontrar padrões e estruturas
- Agrupar dados de forma a criar representações sumarizadas (sumarização de dados)

# Índice

- [Exemplo inicial](#Exemplo-Inicial)
- [K-Means](#K-Means)
 - [Case K-Means Elo7](#Case-Cluster-Usuários-Elo7)
- [Case Elo7 - Cluster Frete](#Case-Elo7---Clustering-de-Frete)
- [Hierarchical Clustering](#Hierarchical-Clustering)
 - [Exercício Prático](#Exercício-prático-Hierarchical-Clustering)
- [Case Elo7 - Motivos de Compra](#Case-Elo7---Motivos-de-Compra)

### Exemplo Inicial
Análise exploratória do comportamento dos usuários do Elo7.

Dataset:
- `tempo` (float): Tempo em segundos que um usuário permanece no site.
- `ticket` (float): Valor gasto em reais no site.

In [ ]:
# Imports usados no curso
%matplotlib inline
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss

In [ ]:
sns.set(style="ticks")
plt.rcParams['figure.figsize'] = (12.0, 8.0)
plt.style.use('seaborn-colorblind')

In [ ]:
# Pasta contendo os dados:
ROOT_FOLDER = os.path.realpath('..')
DATASET_FOLDER = os.path.join(ROOT_FOLDER,'datasets')

In [ ]:
# Leitura dos dados
df_user_elo7 = pd.read_csv(os.path.join(DATASET_FOLDER, 'user_patterns_elo7_dataset.csv'), sep=';')

df_user_elo7.head(5)

#### Análise Exploratória
- Média
- Covariância
- Tendência (Regressão Linear)

In [ ]:
# Valor médio
user_elo7_mean = df_user_elo7.mean().values

# Covariância
user_elo7_cov = np.cov(df_user_elo7.values[:,0], df_user_elo7.values[:,1])

# Tendência - regressão
a, b, r, p, std_err = ss.linregress(df_user_elo7.values[:,0],df_user_elo7.values[:,1])
f = lambda x: a*x + b

In [ ]:
print('- Média: {}'.format(user_elo7_mean))
print('- Covariância: \n{}'.format(user_elo7_cov))
print('- Coeficiente de correlação da regressão: {:.2f}'.format(r))

Esses dados parecem interessantes, mas não são suficientes. Precisamos sempre observar os dados para tirar insights!

In [ ]:
# Vamos plotar o gráfico
df_user_elo7.plot.scatter(x='tempo',y='ticket', alpha=0.5)
plt.xlabel('Tempo (s)')
plt.ylabel('Ticket (R$)')
plt.show()

In [ ]:
### Função auxiliar para plotar a elipse de confiança ###
from matplotlib.patches import Ellipse

def get_confidence_ellipse(x, y, nstd=2):
    def eigsorted(cov):
        vals, vecs = np.linalg.eigh(cov)
        order = vals.argsort()[::-1]
        return vals[order], vecs[:,order]

    cov = np.cov(x, y)
    vals, vecs = eigsorted(cov)
    theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))
    w, h = 2 * nstd * np.sqrt(vals)
    ell = Ellipse(xy=(np.mean(x), np.mean(y)),
                  width=w, height=h,
                  angle=theta, color='red', 
                  fill=False)
    return ell

In [ ]:
# Vamos plotar os dados no gráfico
df_user_elo7.plot.scatter(x='tempo',y='ticket', alpha=0.5)
plt.xlabel('Tempo (s)')
plt.ylabel('Ticket (R$)')

# Média
plt.plot(user_elo7_mean[0], user_elo7_mean[1], '*r', markersize=20)

# 2 desvios padrão
ell = get_confidence_ellipse(x=df_user_elo7.values[:,0],
                             y=df_user_elo7.values[:,1])
ax = plt.gca()
ax.add_patch(ell)

# Tendência
x = np.array([min(df_user_elo7.values[:,0]),max(df_user_elo7.values[:,0])])
plt.plot(x, f(x), '--g')

plt.show()

Há algo estranho nessa análise?
- A análise está matematicamente correta, mas talvez não seja completa;
- Precisamos levar em consideração possíveis grupos diferentes de usuários dentro dos dados. Quantos grupos você vê? Talvez entre 2 e 4 clusters?

Vamos utilizar o famoso algoritmo [KMeans](https://en.wikipedia.org/wiki/K-means_clustering) para encontrar esses clusters. Podemos utilizar a implementação do [sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) para isso.

---
## K-Means

In [ ]:
# TODO
# Importe o módulo do KMeans
from sklearn.cluster import KMeans

# Crie uma instância do K-Means pelo sklearn
# Teste diferentes números de clusters
n = _
kmeans = KMeans(n_clusters=_)

Agora podemos encontrar os clusters.

In [ ]:
X = df_user_elo7.values

kmeans.fit(X)
labels = kmeans.labels_

In [ ]:
plt.scatter(x=df_user_elo7.values[:,0],
            y=df_user_elo7.values[:,1],
            c=labels.astype(np.float),
            cmap='rainbow',
            edgecolor='k')
plt.title('Num clusters: {}'.format(n))
plt.show()

Os clusters estão como esperado? Tem alguma hipótese do porque esses dados estão separados dessa forma?

---
A grande maioria dos algoritmos de aprendizado de máquina se baseiam na noção de distância entre pontos para encontrar relações entre os dados. Essa forma de tratar os problemas é bastante intuitiva e funciona em uma grande gama de cenários. Mas, existe um problema para calcularmos distâncias quando temos diversas variáveis com características diferentes. Para entender um pouco desse problema, note que as escalas dos eixos do gráfico anterior não são iguais. Veja como ficaria o gráfico se colocássemos os eixos com mesma escala:

In [ ]:
plt.scatter(x=df_user_elo7.values[:,0],
            y=df_user_elo7.values[:,1],
            c=labels.astype(np.float),
            cmap='rainbow',
            edgecolor='k')
plt.title('Num clusters: {}'.format(n))
plt.axis('equal')
plt.show()

Por isso devemos **normalizar** os dados! Essa etapa é muito importante e deve ser sempre considerada antes de executar algum algoritmo de aprendizado de máquina que se baseia em distâncias.

In [ ]:
# Vamos normalizar os dados!
from sklearn.preprocessing import StandardScaler

In [ ]:
# E agora plotamos o resultado
n_clusters = range(2,6)

X = df_user_elo7.values

# Dados normalizados
X_scaled = StandardScaler().fit_transform(X)

for n in n_clusters:
    estimator = KMeans(n_clusters=n)
    estimator.fit(X_scaled)
    labels = estimator.labels_
    plt.scatter(x=X_scaled[:,0],
                y=X_scaled[:,1],
                c=labels.astype(np.float),
                cmap='rainbow',
                edgecolor='k')
    plt.title('Num clusters: {}'.format(n))
    plt.show()

Ok... podemos ver que podemos encontrar algumas opções de número de clusters, mas qual é o valor ideal?

#### Escolha do número de clusters

Nós temos diversos métodos para escolher o número ideal de clusters. Alguns deles estão resumidos neste [artigo](https://en.wikipedia.org/wiki/Determining_the_number_of_clusters_in_a_data_set#The_Elbow_Method). O método mais utilizado, entretanto, é o método do "cotovelo" (*elbow method*). 

Mas, antes de falarmos do método do cotovelo, nós precisamos definir o que é um bom cluster. É claro que isso depende de cada caso, mas as seguintes características são desejadas para a maioria dos clusters:
- Dados não muito dispersos -> Inércia
- Dados dentro dos clusters possuem perfil semelhante
- Quantidade aproximadamente uniforme de dados em cada cluster

#### Inércia

A inércia de um cluster é definida como a soma das distâncias quadráticas de cada ponto de um cluster ao seu respectivo centroide, somada através de todos os clusters. Quanto maior é a inércia, maior será a dispersão dos clusters. Portanto, desejamos escolher um número de clusters que nos possibilite ter uma inércia baixa. Simples, mas temos um problema... O mínimo valor de inércia que podemos obter é quando cada ponto do nosso dataset pertence ao seu próprio cluster. Portanto, precisamos escolher um balanço entre baixa inércia e baixo número de clusters. 

Para isso, utilizamos o gráfico de cotovelo. O eixo horizontal do gráfico representa o número de clusters utilizados e o eixo vertical representa a inércia total dos clusters. O número de clusters ideal é definido como o ponto onde o gráfico se aproxima a uma horizontal (como o ponto de encontro do braço e antebraço).

In [ ]:
# Range de valores de clusters que vamos testar
k = range(1,8,1)

# Lista de inércias
inertias = []

# Para cada valor de k, ache a inércia
for i in k:
    # crie a instância
    kmeans = KMeans(n_clusters=i)

    # Treine o modelo
    model = kmeans.fit(X_scaled)

    # Ache a inercia dos clusters
    inertias.append(model.inertia_)
    
plt.plot(k, inertias, '-ob')
plt.xlabel('Clusters')
plt.ylabel('Inertia')
plt.grid()
plt.show()

Qual a sua opinião? Quantos clusters devemos utilizar?

In [ ]:
# TODO

---
## Case Elo7 - Clustering de Frete

Um dos problemas mais complicados do Elo7 é sua dependência dos correios. Nós sofremos muito com a falta de alternativas para dar aos nossos clientes (compradores e vendedores), já que o serviço dos correios além de caro, é também instável. 

Para tentar resolver esse problema, o time de Data Science do Elo7 foi chamado para tentar encontrar alguma alternativa. Após algumas conversas, nós levantamos a possibilidade de utilizarmos serviços de entrega independentes dos correios. Mas, o problema é que esses serviços necessitam de um volume grande de encomendas por ponto de coleta, o que não é o caso para a maioria dos vendedores cadastrados no Elo7. 

Uma possível solução seria encontrar pontos de coleta que pudessem agregar pedidos de vários vendedores e enviar de uma vez só com um desses serviços alternativos. Mas, como obtemos a localização desses pontos de coleta? Podemos aplicar um algoritmo de clustering nas rotas de frete mais frequentes!

Vamos tentar analisar os dados e verificar o que conseguimos obter. O dataset a seguir contém pares de endereços de origem e destino de entregas realizadas apenas na cidade de São Paulo em um curto intervalo de tempo.

In [ ]:
df_route = pd.read_csv(os.path.join(DATASET_FOLDER, 'route_clustering_elo7_dataset.csv'), sep=';')

df_route.head()

Para facilitar os cálculos de distância, as latitudes e longitudes dos locais já foram realizados.

Vamos agora formar nosso vetor de features contendo as posições geográficas das nossas rotas.

*Dica: Será que é necessário normalizar as features?

In [ ]:
# TODO
X = df_route[['latitude_origem','longitude_origem','latitude_destino','longitude_destino']].values
X_scaled = _ # ?

Quantos clusters vamos utilizar? (Obs: Podemos aplicar o método do cotovelo para descobrir.)

In [ ]:
# TODO

Agora podemos iniciar o algoritmo de clustering.

In [ ]:
# TODO
n = _
kmeans = KMeans(n_clusters=n)

In [ ]:
# TODO
clusters = _

A análise da quantidade de ítens em cada cluster é sempre uma boa prática. Clusters desbalanceados são um sinal de que os dados não foram bem separados.

In [ ]:
cluster, count = np.unique(clusters, return_counts=True)
for l, c in zip(cluster,count):
    print('Cluster {}: {}'.format(l,c))

Vamos ver os gráficos para analisar qualitativamente os resultados.

In [ ]:
labels = kmeans.labels_

ax1 = plt.subplot(1,2,1)
ax1.set_title('Origem')
plt.scatter(x=X[:,0],
            y=X[:,1],
            c=labels, 
            edgecolor='k',
            cmap='rainbow')
ax1.set_xlim((-23.4,-23.9))

ax2 = plt.subplot(1,2,2)
ax2.set_title('Destino')
plt.scatter(x=X[:,2],
            y=X[:,3],
            c=labels, 
            edgecolor='k',
            cmap='rainbow')
ax2.set_xlim((-23.4,-23.9))

plt.show()

O que achou? É possível perceber clusters bem definidos? Será que podemos utilizar esses clusters para resolver nossos problemas de frete?

---
### Exemplo Prático - Kaggle NYC Taxi Trip Duration

Dados:
- id - a unique identifier for each trip
- vendor_id - a code indicating the provider associated with the trip record
- pickup_datetime - date and time when the meter was engaged
- dropoff_datetime - date and time when the meter was disengaged
- passenger_count - the number of passengers in the vehicle (driver entered value)
- pickup_longitude - the longitude where the meter was engaged
- pickup_latitude - the latitude where the meter was engaged
- dropoff_longitude - the longitude where the meter was disengaged
- dropoff_latitude - the latitude where the meter was disengaged
- store_and_fwd_flag - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip
- trip_duration - duration of the trip in seconds

![iris](https://cdn.civitatis.com/estados-unidos/nueva-york/galeria/thumbs/taxi-nueva-york.jpg)

*Solução retirada do github de [juifa-tsai](https://github.com/juifa-tsai/NYC_Taxi_Trip_Duration).

Vamos realizar uma análise dos dados.

In [ ]:
df_taxi = pd.read_csv(os.path.join(DATASET_FOLDER,'nyc_trip_duration_dataset.csv'))

In [ ]:
df_taxi.head(5)

Podemos utilizar diversas abordagens para analisar os dados. Vamos tentar verificar os dados de localização dos passageiros.

In [ ]:
df_map = df_taxi[['pickup_longitude','pickup_latitude', 'dropoff_longitude','dropoff_latitude']]
df_pick = df_map[['pickup_longitude','pickup_latitude']]
df_drop = df_map[['dropoff_longitude','dropoff_latitude']]

Vamos visualizar os dados.

In [ ]:

def plot_map(df, zoom=0.9):
    cutmap = zoom/100

    x = df['pickup_longitude']
    y = df['pickup_latitude']
    x_max, x_min = x.quantile(1-cutmap), x.quantile(cutmap)
    y_max, y_min = y.quantile(1-cutmap), y.quantile(cutmap)
    
    x_plot = x[(x>x_min) & (x<x_max) & (y<y_max) & (y>y_min)]
    y_plot = y[(x>x_min) & (x<x_max) & (y<y_max) & (y>y_min)]
    plt.scatter(x=x_plot, y=y_plot, s=5, alpha=0.3)
    plt.tick_params(labelsize=18)
    plt.title('Pickup', fontsize=18 )
    plt.xlabel('Longitude', fontsize=18)
    plt.ylabel('Latitude',  fontsize=18)
    plt.show()

plot_map(df_taxi)

A distribuição dos dados é bem interessante. Podemos verificar que existe uma concentração grande de pontos dentro da ilha de Manhattan, o que é esperado.

Como segundo passo da análise dos dados, nós podemos tentar enriquecê-los utilizando técnicas de feature engineering e clustering. Vamos explorar o segundo em seguida.

O racional de utilizar clustering para análise exploratória e feature engineering é o fato de encontrar estruturas implícitas nos dados. Por exemplo, se tentássemos observar cada passageiro individualmente, talvez teríamos dificuldade em encontrar um padrão nos dados. Mas, é intuitivo pensar que passageiros semelhantes (mesma localização, horário etc) possam ser agrupados e tratados como um só. Assim, podemos tratar os dados por grupos controlados de passageiros, ao invés de cada indivíduo.

Vamos tentar encontrar clusters nos dados de início da corrida de taxi.

In [ ]:
kmeans = KMeans(n_clusters=20)

In [ ]:
X_kmeans = kmeans.fit_predict(df_pick)
df_pick['zone']  = X_kmeans

In [ ]:
def draw_map_zone( df, x_name, y_name, z_name, name, zoom=0.9, cluster=None ):

    x = df[x_name]
    y = df[y_name]
    z = df[z_name]

    cutmap = zoom/100
    x_max, x_min = x.quantile(1-cutmap), x.quantile(cutmap)
    y_max, y_min = y.quantile(1-cutmap), y.quantile(cutmap)
    
    zones = np.unique(z[(x>x_min) & (x<x_max) & (y<y_max) & (y>y_min)])

    #cmap = plt.get_cmap('spectral') 
    cmap = plt.get_cmap('winter') 
    colors = [cmap(i) for i in np.linspace(0, 1, len(zones))]

    for i, zone in enumerate(zones):       
        plt.scatter( x=x[ (z==zone) & (x>x_min) & (x<x_max) & (y<y_max) & (y>y_min) ], 
                     y=y[ (z==zone) & (x>x_min) & (x<x_max) & (y<y_max) & (y>y_min) ], 
                     s=5, alpha=0.3, c=colors[i])
        if cluster:
            plt.text( cluster.cluster_centers_[zone,0], cluster.cluster_centers_[zone,1], str(zone), fontsize = 12, color='r')

    plt.tick_params(labelsize=18)
    plt.title(name, fontsize=18 )
    plt.xlabel('Longitude', fontsize=18)
    plt.ylabel('Latitude',  fontsize=18)


plt.figure(figsize=(17,15))
draw_map_zone(df_pick, 'pickup_longitude', 'pickup_latitude', 'zone', 'Pickup', cluster=kmeans)
plt.show()

Não por acaso, os clusters encontrados se assemelham aos bairros de Nova Iorque. Esses clusters agora podem ser utilizados de diversas formas:
- Podemos explorar a distribuição das outras features dentro de cada um dos clusters. Assim poderemos ver o quanto cada região se diferença das outras.
- Podemos também utilizar agora as labels obtidas pelo algoritmo de clustering como entrada de outros algoritmos de machine learning. Essa técnica é muito utilizada para melhorar a precisão dos algoritmos de regressão e classificação.

Vamos tentar utilizar clustering na solução encontrada na [aula 19](https://github.com/somostera/tera-datascience-out2018/blob/master/19-decision-trees/notebooks/Gabarito%20Aula%2019%20-%20%C3%81rvores%20de%20Decis%C3%A3o.ipynb) para verificar se conseguimos aumentar a precisão do regressor.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import KFold

In [ ]:
kfold = KFold(n_splits=5,random_state=0)

df_taxi_cluster = df_taxi.copy()

kmeans = KMeans(n_clusters=100)
X_kmeans = kmeans.fit_predict(df_map)
df_taxi_cluster['zone'] = X_kmeans

x = df_taxi.drop(['trip_duration', 'id', 'pickup_datetime', 'dropoff_datetime', 'store_and_fwd_flag'], axis=1)
y = df_taxi['trip_duration']

x_cluster = df_taxi_cluster.drop(['trip_duration', 'id', 'pickup_datetime', 'dropoff_datetime', 'store_and_fwd_flag'], axis=1)

reg = DecisionTreeRegressor()

In [ ]:
def make_cv_prediction(x,y,train_index,test_index,reg):
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    reg.fit(x_train, y_train)
    y_pred = reg.predict(x_test)
    return y_pred, y_test

def rmsle(y_test, y_pred):
    return np.sqrt(mean_squared_log_error(y_test,y_pred))

rmsle_cv_default = []
rmsle_cv_cluster = []
for train_index, test_index in kfold.split(x,y):
    y_pred, y_test = make_cv_prediction(x,y,train_index,test_index,reg)
    rmsle_cv_default.append(rmsle(y_test, y_pred))
    
    y_pred_cluster, y_test = make_cv_prediction(x_cluster,y,train_index,test_index,reg)
    rmsle_cv_cluster.append(rmsle(y_test, y_pred_cluster))

In [ ]:
print('Default: {:.4f}'.format(np.mean(rmsle_cv_default)))
print('Cluster: {:.4f}'.format(np.mean(rmsle_cv_cluster)))

Pode-se notar um ligeiro aumento de precisão do estimador ao utilizar o cluster.

---
## Hierarchical Clustering

Vamos agora aprender sobre outro método de clustering: [**Hierarchical Clustering**](https://en.wikipedia.org/wiki/Hierarchical_clustering). Como o nome mesmo diz, ele utiliza o conceito de *hierarquia* para construir os clusters. Existem duas principais variações do algoritmo: aglomerativo e por divisão. O primeiro é mais usado na prática. O passo a passo do algoritmo é apresentado abaixo:

- Primeiro colocamos todos as observações em clusters próprios (individuais);
- Depois, iterativamente procuramos os clusters mais próximos\* e agrupamos eles em um novo cluster;
- Repetimos o passo anterior até formarmos um único cluster com todas as observações.

\*Obs: A definição de distância (ou similaridade) entre clusters depende do tipo de métrica de distância (Euclidiana, Manhattan, cosseno etc) e ligação (Ward, simples, completa etc).

Como podemos ver no algoritmo, o objetivo é a criação de um grande cluster que agrupe todos os dados. Nós podemos visualizar esse histórico de agrupamentos a partir de um [dendrograma](https://en.wikipedia.org/wiki/Dendrogram). A então criação de clusters mais granulares depende da região de similaridade que se deseja realizar o corte.

Vamos aplicar o método de Hierarchical Clustering no dataset de usuários do Elo7.

In [ ]:
from sklearn.preprocessing import StandardScaler

df_user_elo7 = pd.read_csv(os.path.join(DATASET_FOLDER, 'user_patterns_elo7_dataset.csv'), sep=';')

X = df_user_elo7.values

# Dados normalizados
X_scaled = StandardScaler().fit_transform(X)

In [ ]:
# Importe os métodos linkage (Hierarchical Clustering) e dendrogram
from scipy.cluster.hierarchy import linkage, dendrogram

O scikit-learn possui um método próprio para o algoritmo de [Hierarchical Clustering](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering). Entretanto, ele não nos permite visualizar facilmente o dendrograma final. Por isso, vamos utilizar a versão do scipy.

In [ ]:
# TODO
# Vamos escolher a métrica de distância:
distance = _ # 'euclidean'|'cityblock'|'cosine'...
# Agora o tipo de ligação
linkage_type = _ # 'single'|'complete'|'average'|'ward'...

# Vamos aplicar o método linkage
Y = linkage(X_scaled, method=linkage_type, metric=distance)

In [ ]:
Y.shape

In [ ]:
# Vamos visualizar o dendrograma
plt.figure(figsize=(16,10))
dendrogram(Y,
           leaf_rotation=90,
           leaf_font_size=6,
)
plt.show()

O que achou? Teste outros valores de distância e tipo de ligação para verificar as diferenças nos resultados!

O dendrograma nos permite verificar qual é o número de clusters que vamos escolher ao final. Além disso, podemos verificar se a distância e o tipo de ligação foram bem escolhidos.

O que precisamos fazer agora é escolher o número de clusters. Podemos utilizar o mesmo método do cotovelo para esse objetivo, mas, na prática, podemos apenas visualizar qual é a região que possui maior distância entre aglutinações. Outros trabalhos ainda utilizam um coeficiente de [Correlação Cofenética](https://en.wikipedia.org/wiki/Cophenetic_correlation) para encontrar uma boa posição de corte no dendrograma.

Para essa tarefa nós podemos usar o método [`fcluster`](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.cluster.hierarchy.fcluster.html) do scipy. Ele nos permite realizar um corte na árvore de clustering gerada pelo Hierarchical Clustering.

In [ ]:
from scipy.cluster.hierarchy import fcluster

In [ ]:
# TODO
# Vamos gerar os rótulos para os clustes
num_clusters = _ # escolha o número de clusters que deseja
labels = fcluster(Y, num_clusters ,criterion='maxclust')

In [ ]:
plt.scatter(x=X_scaled[:,0],
            y=X_scaled[:,1],
            c=labels.astype(np.float),
            cmap='rainbow',
            edgecolor='k')
plt.title('Num clusters: {}'.format(num_clusters))
plt.show()

---
## Case Elo7 - Clustering de Frete

Vamos tentar aplicar o mesmo algoritmo para o problema de cluster de frete.

In [ ]:
df_route = pd.read_csv(os.path.join(DATASET_FOLDER, 'route_clustering_elo7_dataset.csv'), sep=';')

df_route.head()

In [ ]:
X = df_route[['latitude_origem','longitude_origem','latitude_destino','longitude_destino']].values
X_scaled = StandardScaler().fit_transform(X)

In [ ]:
# TODO
# Vamos escolher a métrica de distância:
distance = _
# Agora o tipo de ligação
linkage_type = _

# Vamos aplicar o método linkage
Y = _

In [ ]:
# Vamos visualizar o dendrograma
plt.figure(figsize=(16,10))
dendrogram(Y,
           leaf_rotation=90,
           leaf_font_size=6,
)
plt.show()

In [ ]:
# TODO
# Vamos gerar os rótulos para os clustes
num_clusters = _
labels = _

In [ ]:
ax1 = plt.subplot(1,2,1)
ax1.set_title('Origem')
plt.scatter(x=X[:,0],
            y=X[:,1],
            c=labels, 
            edgecolor='k',
            cmap='rainbow')
ax1.set_xlim((-23.4,-23.9))

ax2 = plt.subplot(1,2,2)
ax2.set_title('Destino')
plt.scatter(x=X[:,2],
            y=X[:,3],
            c=labels, 
            edgecolor='k',
            cmap='rainbow')
ax2.set_xlim((-23.4,-23.9))

plt.show()

---
# Case Elo7 - Motivos de Compra

Os compradores do Elo7 são incentivados a indicar o motivo da compra de determinado produto no seu marketplace. Esses motivos nos ajudam a entender melhor o **momento** de compra do usuário. O dataset apresentado a seguir contém um subset desses motivos de compra.

In [ ]:
df_reason = pd.read_csv(os.path.join(DATASET_FOLDER, 'purchase_reason_elo7_dataset.csv'), sep=';')

df_reason.head(10)

Existem muitos tipos possíveis de motivos de compra, mas será que nós podemos encontrar algum padrão neles? Me parece um problema clássico de **clustering**.

Vamos utilizar o Tf-Idf para criar o embedding dos motivos de compra e o K-Means para encontrar clusters.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_df=0.9, max_features=5000, sublinear_tf=True, use_idf=True)

Cria a matriz de embeddings.

In [ ]:
X = tfidf.fit_transform(df_reason['reason'].values)

Como escolher o número de clusters? Vamos utilizar o gráfico de inércias. (Obs: outra possibilidade é avaliar o ["silhouette score"](http://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py)).

In [ ]:
# Range de valores de clusters que vamos testar
k = range(10,200,20)

# Lista de inércias
inertias = []

# Para cada valor de k, ache a inércia
for i in k:
    # crie a instância
    kmeans = KMeans(n_clusters=i)

    # Treine o modelo
    model = kmeans.fit(X)

    # Ache a inercia dos clusters
    inertias.append(model.inertia_)
    
plt.plot(k, inertias, '-ob')
plt.xlabel('Clusters')
plt.ylabel('Inertia')
plt.grid()
plt.show()

Inicializa o K-Means com a quantidade de clusters que escolhemos a partir do gráfico.

In [ ]:
# TODO
n = _
kmeans = KMeans(n_clusters=n)

Treine o modelo K-Means.

In [ ]:
kmeans.fit(X)

Encontra os clusters para cada motivo de compra.

In [ ]:
# TODO
labels = _

Vamos agora visualizar os clusters criados.

In [ ]:
# Crie um novo dataframe com os labels dos clusters
df = pd.DataFrame({'reason': df_reason['reason'], 'labels': labels})

df.head()

Vamos verificar a distribuição de motivos em cada cluster. Quanto mais desbalanceado, pior.

In [ ]:
df.groupby('labels').size()

Podemos visualizar alguns exemplos de clusters gerados pelo K-Means.

In [ ]:
for idx in range(50):
    idx_labels = df[df['labels']==idx]['reason'].unique()
    print('- Cluster {}:'.format(idx + 1))
    for i in np.random.choice(idx_labels, min(len(idx_labels), 10), replace=False):
        print(' '*5, i)
    print()

Qual é o resultado dos clusters gerados? Podemos avançar um pouco e verificar se existe alguma relação de hierarquia entre os motivos de compra.

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

# TODO
Y = linkage(_, method=_, metric=_)

In [ ]:
# Obtém aleatoriamente um dos motivos para representar o cluster
titles = df.groupby('labels').apply(lambda x: np.random.choice(list(x['reason'])))

In [ ]:
# plota o dendrograma
plt.figure(figsize=(16,10))
dendrogram(Y,
           labels=titles.values,
           leaf_rotation=90,
           leaf_font_size=14,
)
plt.show()

Qual foi o resultado? O que você faria para melhorar o resultado obtido?

---
# Case Elo7 - Subcategorias Automáticas

Vamos para mais um case real do Elo7!

Esse case é um dos trabalhos mais recentes do time de Data Science do Elo7. De fato, é um trabalho ainda em aberto e qualquer sugestão de melhorias é bem vinda! =)

- O problema:
O Elo7 possui uma árvore de categorias dividida em N1 e N2. O primeiro nível (N1) contém as categorias "alto nível" do site. São as categorias mais genéricas do marketplace- ou, pelo menos, é assim gostaríamos que fosse. As categorias N2, ou subcategorias, são as possíveis extensões dos nós das categorias N1. Podemos perceber que a árvore é extremamente limitada e isso é um problema grave não só para os compradores, que não conseguem navegar nas nossas categorias, mas também para os vendedores, que não conseguem categorizar bem seus produtos. A solução para esse problema seria uma árvore de categorias com maior "granularidade", ou seja, que consiga expandir além dos 2 níveis e ter mais subcategorias.

- O que o time de Data Science tem a ver com essa história? 

Bom, gerar uma nova árvore de categoria pode ser uma tarefa bastante monótona e cansativa. Provavelmente deve haver algum jeito de encontrar bons agrupamentos de produtos que pudessem servir como uma nova subcategoria. Talvez algum método de clustering que utilize como features o conteúdo dos produtos pode gerar algum resultado interessante.

- O experimento:

O dataset a seguir possui um subconjunto de produtos que foram categorizados na categoria N1 "Casamento". Escolhemos esse conjunto de dados para iniciar nossos trabalhos, porque assim temos mais controle sobre nossos resultados. E, também, porque é uma das categorias mais importantes do marketplace.

Para essa tarefa, vamos utilizar apenas o título e uma parte da descrição do produto (aprox. 140 caracteres) como features de entrada.

Vamos analisar os dados!

In [ ]:
df_cat = pd.read_csv(os.path.join(DATASET_FOLDER, 'subcategory_elo7_dataset.csv'), sep=';')

df_cat.head(10)

Vamos criar uma coluna com as features que vamos incluir no nosso modelo de aprendizagem.
Esse vetor de features será o título + descrição do produto. Para compensar a quantidade de palavras do título em relação a descrição, vamos repetir o título duas vezes.

In [ ]:
df_cat['title_desc'] = (df_cat['title'] + ' ')*2 + df_cat['short_description']

df_cat.head(10)

Tente encontrar as subcategorias dos produtos da categoria "casamento". Lembre-se de que não queremos apenas aumentar o número de subcategorias do segundo nível (N2), mas também aumentar a profundidade da nossa árvore de categorias (N3, N4 ...).

Para criar nossa matriz de features, nós vamos utilizar o Tf-Idf.

In [ ]:
tfidf = TfidfVectorizer(max_df=0.9, max_features=10000, sublinear_tf=True, use_idf=True)

In [ ]:
X = tfidf.fit_transform(df_cat['title_desc'].values)

Novamente, precisamos definir o número de clusters. Podemos utilizar o método do gráfico de inércias.

(Obs: O cálculo pode levar muito tempo para ser executado. Assuma que o valor escolhido é 40)

In [ ]:
# Range de valores de clusters que vamos testar
k = range(10,100,10)

# Lista de inércias
inertias = []

# Para cada valor de k, ache a inércia
for i in k:
    # crie a instância
    kmeans = KMeans(n_clusters=i)

    # Treine o modelo
    model = kmeans.fit(X)

    # Ache a inercia dos clusters
    inertias.append(model.inertia_)
    
plt.plot(k, inertias, '-ob')
plt.xlabel('Clusters')
plt.ylabel('Inertia')
plt.grid()
plt.show()

In [ ]:
# TODO
kmeans = _

In [ ]:
# TODO
labels = _

In [ ]:
df = pd.DataFrame({'title': df_cat['title'], 'labels': labels})

In [ ]:
df.groupby('labels').size()

In [ ]:
for idx in range(len(df['labels'].unique())):
    idx_labels = df[df['labels']==idx]['title'].unique()
    print('- Cluster {}:'.format(idx + 1))
    for i in np.random.choice(idx_labels, min(len(idx_labels), 10), replace=False):
        print(' '*5, i)
    print()

In [ ]:
# TODO
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

Y = linkage(_, method=_, metric=_)

In [ ]:
titles = df.groupby('labels').apply(lambda x: np.random.choice(list(x['title'])))

In [ ]:
plt.figure(figsize=(16,10))
dendrogram(Y,
           labels=titles.values,
           leaf_rotation=90,
           leaf_font_size=14,
)
plt.show()